MODEL BUILDING FOR PRICE OPTIMIZATION

BUSINESS OBJECTIVE: Build ML model to predict profit at different price points
TECHNICAL OBJECTIVE: Compare models, select best for production deployment

MODEL SELECTION CRITERIA:
1. Prediction accuracy (R², MAE, RMSE)
2. Interpretability (can we explain to stakeholders?)
3. Speed (real-time pricing decisions)
4. Robustness (works across all products/segments)

MODELS TO TEST:
1. Linear Regression - Baseline, highly interpretable
2. Ridge Regression - Regularized linear, prevents overfitting
3. Random Forest - Non-linear, feature importance, robust
4. Gradient Boosting - Best accuracy, industry standard
5. XGBoost - Production-ready, fast, accurate


In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import joblib
import warnings
warnings.filterwarnings('ignore')


# ============================================================================
# LOAD DATA AND FEATURES
# ============================================================================
print("="*80)
print("1. LOAD DATA")
print("="*80)
print()

df = pd.read_csv('lab_equipment_pricing_features.csv')
print(f"✓ Loaded: {len(df):,} records with {df.shape[1]} columns")

# Load feature metadata
with open('feature_metadata.json', 'r') as f:
    feature_metadata = json.load(f)

all_features = feature_metadata['all_features']
target = feature_metadata['target']

print(f"✓ Features: {len(all_features)}")
print(f"✓ Target: {target}")
print()

1. LOAD DATA

✓ Loaded: 10,000 records with 44 columns
✓ Features: 27
✓ Target: profit



In [2]:
# ============================================================================
# PREPARE TRAIN/TEST SPLIT
# ============================================================================
print("="*80)
print("2. TRAIN/TEST SPLIT STRATEGY")
print("="*80)
print()

print("WHY TIME-BASED SPLIT:")
print("  - Can't use random split (data leakage risk)")
print("  - Must simulate real scenario: train on past, predict future")
print("  - Business: Model must work on upcoming quarters")
print()

# Sort by date
df_sorted = df.sort_values('date').reset_index(drop=True)

# Remove rows with NaN in features (from rolling calculations)
df_model = df_sorted[all_features + [target]].dropna()
print(f"Rows after removing NaN: {len(df_model):,}")

X = df_model[all_features]
y = df_model[target]

# Time-based split: 80% train, 20% test
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"\nTrain set: {len(X_train):,} records (80%)")
print(f"Test set: {len(X_test):,} records (20%)")
print(f"Train target mean: ${y_train.mean():,.0f}")
print(f"Test target mean: ${y_test.mean():,.0f}")
print()


2. TRAIN/TEST SPLIT STRATEGY

WHY TIME-BASED SPLIT:
  - Can't use random split (data leakage risk)
  - Must simulate real scenario: train on past, predict future
  - Business: Model must work on upcoming quarters

Rows after removing NaN: 10,000

Train set: 8,000 records (80%)
Test set: 2,000 records (20%)
Train target mean: $467,910
Test target mean: $452,682



In [3]:
# ============================================================================
# FEATURE SCALING
# ============================================================================
print("="*80)
print("3. FEATURE SCALING")
print("="*80)
print()

print("WHY SCALING:")
print("  - Linear models sensitive to feature scales")
print("  - Tree models don't need scaling (but doesn't hurt)")
print("  - Speeds up convergence")
print()

# 2. Identify Columns
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
# 3. Define Preprocessors
# Transformation for numerical data (scaling)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Transformation for categorical data (encoding)
# handle_unknown='ignore' prevents an error if a new category appears in test data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply the transformations
X_train_scaled  = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)
all_features_expanded = preprocessor.get_feature_names_out()

print("✓ Features scaled using StandardScaler")
print(f"  Mean: ~0, Std: ~1 for all features")
print()


3. FEATURE SCALING

WHY SCALING:
  - Linear models sensitive to feature scales
  - Tree models don't need scaling (but doesn't hurt)
  - Speeds up convergence

✓ Features scaled using StandardScaler
  Mean: ~0, Std: ~1 for all features



In [4]:
# ============================================================================
# MODEL 1: LINEAR REGRESSION (BASELINE)
# ============================================================================
print("="*80)
print("4. MODEL 1: LINEAR REGRESSION (BASELINE)")
print("="*80)
print()

print("WHY LINEAR REGRESSION:")
print("  Business: Highly interpretable, shows feature importance")
print("  Technical: Fast, simple, good baseline")
print("  Limitation: Assumes linear relationships")
print()

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_train_lr = lr_model.predict(X_train_scaled)
y_pred_test_lr = lr_model.predict(X_test_scaled)

# Metrics
train_r2_lr = r2_score(y_train, y_pred_train_lr)
test_r2_lr = r2_score(y_test, y_pred_test_lr)
test_mae_lr = mean_absolute_error(y_test, y_pred_test_lr)
test_rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_test_lr))
test_mape_lr = mean_absolute_percentage_error(y_test, y_pred_test_lr)

print("Performance Metrics:")
print(f"  Train R²: {train_r2_lr:.4f}")
print(f"  Test R²: {test_r2_lr:.4f}")
print(f"  Test MAE: ${test_mae_lr:,.0f}")
print(f"  Test RMSE: ${test_rmse_lr:,.0f}")
print(f"  Test MAPE: {test_mape_lr:.1%}")
print()

# Feature importance (coefficients)
feature_importance_lr = pd.DataFrame({
    'feature': all_features_expanded,
    'coefficient': lr_model.coef_
}).sort_values('coefficient', key=abs, ascending=False)

print("Top 10 Most Important Features (by coefficient magnitude):")
print(feature_importance_lr.head(10).to_string(index=False))
print()

print("BUSINESS INTERPRETATION:")
top_feature = feature_importance_lr.iloc[0]
print(f"  Most influential: {top_feature['feature']} (coef: {top_feature['coefficient']:.2f})")
print(f"  → Each unit increase in {top_feature['feature']} changes profit by ${top_feature['coefficient']:.2f}")
print()


4. MODEL 1: LINEAR REGRESSION (BASELINE)

WHY LINEAR REGRESSION:
  Business: Highly interpretable, shows feature importance
  Technical: Fast, simple, good baseline
  Limitation: Assumes linear relationships

Performance Metrics:
  Train R²: 0.8636
  Test R²: 0.8532
  Test MAE: $131,590
  Test RMSE: $171,931
  Test MAPE: 275.3%

Top 10 Most Important Features (by coefficient magnitude):
                feature    coefficient
   num__inventory_level -617285.352717
     num__inventory_pct  611923.355536
       num__price_ma_7d  253838.593420
      num__price_ma_30d  131500.702226
   num__segment_encoded   99820.713118
    num__price_momentum   35110.044630
 num__is_academic_start   34409.616418
num__is_summer_slowdown  -33408.598185
         num__qty_ma_7d   15778.984799
     cat__season_Summer  -13839.388305

BUSINESS INTERPRETATION:
  Most influential: num__inventory_level (coef: -617285.35)
  → Each unit increase in num__inventory_level changes profit by $-617285.35



In [5]:
# ============================================================================
# MODEL 2: RIDGE REGRESSION (REGULARIZED)
# ============================================================================
print("="*80)
print("5. MODEL 2: RIDGE REGRESSION (REGULARIZED)")
print("="*80)
print()

print("WHY RIDGE:")
print("  Business: Same interpretability as linear, but more robust")
print("  Technical: L2 regularization prevents overfitting")
print("  Use case: When we have many correlated features")
print()

ridge_model = Ridge(alpha=10.0)  # Regularization strength
ridge_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_train_ridge = ridge_model.predict(X_train_scaled)
y_pred_test_ridge = ridge_model.predict(X_test_scaled)

# Metrics
train_r2_ridge = r2_score(y_train, y_pred_train_ridge)
test_r2_ridge = r2_score(y_test, y_pred_test_ridge)
test_mae_ridge = mean_absolute_error(y_test, y_pred_test_ridge)
test_rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_test_ridge))
test_mape_ridge = mean_absolute_percentage_error(y_test, y_pred_test_ridge)

print("Performance Metrics:")
print(f"  Train R²: {train_r2_ridge:.4f}")
print(f"  Test R²: {test_r2_ridge:.4f}")
print(f"  Test MAE: ${test_mae_ridge:,.0f}")
print(f"  Test RMSE: ${test_rmse_ridge:,.0f}")
print(f"  Test MAPE: {test_mape_ridge:.1%}")
print()


5. MODEL 2: RIDGE REGRESSION (REGULARIZED)

WHY RIDGE:
  Business: Same interpretability as linear, but more robust
  Technical: L2 regularization prevents overfitting
  Use case: When we have many correlated features

Performance Metrics:
  Train R²: 0.8636
  Test R²: 0.8533
  Test MAE: $131,553
  Test RMSE: $171,869
  Test MAPE: 274.9%



In [6]:
# ============================================================================
# MODEL 3: RANDOM FOREST (NON-LINEAR)
# ============================================================================
print("="*80)
print("6. MODEL 3: RANDOM FOREST")
print("="*80)
print()

print("WHY RANDOM FOREST:")
print("  Business: Captures non-linear relationships (price curves aren't straight lines)")
print("  Technical: Robust, handles outliers well, provides feature importance")
print("  Use case: When relationships are complex")
print()

# 1. Define the numerical transformer to just "pass through" the data
#    This explicitly tells the transformer to leave numerical columns alone.
numeric_passthrough = 'passthrough'

rf_preprocessor = ColumnTransformer(
    transformers=[
        # Numerical features are passed through without scaling
        ('num', numeric_passthrough, numerical_features),
        # Categorical features are One-Hot Encoded
        ('cat', categorical_transformer, categorical_features)
    ],
    # Important: If you have other columns (like IDs) that aren't in these lists,
    # remainder='passthrough' will include them as well.
    remainder='drop' 
)

# Apply the transformation
X_train_rfprocessed = rf_preprocessor.fit_transform(X_train)
X_test_rfprocessed = rf_preprocessor.transform(X_test)
all_rf_features=rf_preprocessor.get_feature_names_out()

6. MODEL 3: RANDOM FOREST

WHY RANDOM FOREST:
  Business: Captures non-linear relationships (price curves aren't straight lines)
  Technical: Robust, handles outliers well, provides feature importance
  Use case: When relationships are complex



In [7]:
X_test_rfprocessed

array([[ 1.32600e+01,  5.00000e+00,  2.00000e+00, ...,  1.00000e+00,
         0.00000e+00,  0.00000e+00],
       [ 9.02760e+02,  5.00000e+00,  2.00000e+00, ...,  1.00000e+00,
         0.00000e+00,  0.00000e+00],
       [-1.16023e+03,  5.00000e+00,  2.00000e+00, ...,  1.00000e+00,
         0.00000e+00,  0.00000e+00],
       ...,
       [ 3.66600e+01,  1.20000e+01,  4.00000e+00, ...,  0.00000e+00,
         0.00000e+00,  1.00000e+00],
       [-7.37830e+02,  1.20000e+01,  4.00000e+00, ...,  0.00000e+00,
         0.00000e+00,  1.00000e+00],
       [ 9.66000e+00,  1.20000e+01,  4.00000e+00, ...,  0.00000e+00,
         0.00000e+00,  1.00000e+00]], shape=(2000, 32))

In [8]:



rf_model = RandomForestRegressor(
    n_estimators=100,        # Number of trees
    max_depth=15,            # Prevent overfitting
    min_samples_split=20,    # Minimum samples to split
    min_samples_leaf=10,     # Minimum samples per leaf
    max_features='sqrt',     # Features per split
    random_state=42,
    n_jobs=-1               # Use all CPU cores
)

print("Training Random Forest...")
rf_model.fit(X_train_rfprocessed, y_train)  # Trees don't need scaling

# Predictions
y_pred_train_rf = rf_model.predict(X_train_rfprocessed)
y_pred_test_rf = rf_model.predict(X_test_rfprocessed)

# Metrics
train_r2_rf = r2_score(y_train, y_pred_train_rf)
test_r2_rf = r2_score(y_test, y_pred_test_rf)
test_mae_rf = mean_absolute_error(y_test, y_pred_test_rf)
test_rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_test_rf))
test_mape_rf = mean_absolute_percentage_error(y_test, y_pred_test_rf)

print("\nPerformance Metrics:")
print(f"  Train R²: {train_r2_rf:.4f}")
print(f"  Test R²: {test_r2_rf:.4f}")
print(f"  Test MAE: ${test_mae_rf:,.0f}")
print(f"  Test RMSE: ${test_rmse_rf:,.0f}")
print(f"  Test MAPE: {test_mape_rf:.1%}")
print()

# Feature importance
feature_importance_rf = pd.DataFrame({
    'feature': all_rf_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features (by RF importance):")
print(feature_importance_rf.head(10).to_string(index=False))
print()




Training Random Forest...



Performance Metrics:
  Train R²: 0.9609
  Test R²: 0.9456
  Test MAE: $61,488
  Test RMSE: $104,623
  Test MAPE: 17.4%

Top 10 Most Important Features (by RF importance):
                      feature  importance
            num__price_ma_30d    0.283730
             num__price_ma_7d    0.209109
num__price_inventory_pressure    0.150832
         num__product_encoded    0.143436
         num__segment_encoded    0.077230
   num__price_diff_competitor    0.039549
               num__qty_ma_7d    0.023560
              num__qty_ma_30d    0.010229
      num__is_summer_slowdown    0.007854
           cat__season_Summer    0.006949



In [9]:
# ============================================================================
# MODEL 4: GRADIENT BOOSTING (BEST PERFORMANCE)
# ============================================================================
print("="*80)
print("7. MODEL 4: GRADIENT BOOSTING")
print("="*80)
print()

print("WHY GRADIENT BOOSTING:")
print("  Business: Industry standard for pricing, high accuracy")
print("  Technical: Sequential learning, corrects previous errors")
print("  Use case: When we need best possible predictions")
print()

gb_model = GradientBoostingRegressor(
    n_estimators=200,        # More trees = better but slower
    learning_rate=0.05,      # Smaller = more conservative
    max_depth=5,             # Tree depth
    min_samples_split=20,
    min_samples_leaf=10,
    subsample=0.8,           # Use 80% of data per tree
    max_features='sqrt',
    random_state=42
)

print("Training Gradient Boosting...")
gb_model.fit(X_train_rfprocessed, y_train)

# Predictions
y_pred_train_gb = gb_model.predict(X_train_rfprocessed)
y_pred_test_gb = gb_model.predict(X_test_rfprocessed)

# Metrics
train_r2_gb = r2_score(y_train, y_pred_train_gb)
test_r2_gb = r2_score(y_test, y_pred_test_gb)
test_mae_gb = mean_absolute_error(y_test, y_pred_test_gb)
test_rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_test_gb))
test_mape_gb = mean_absolute_percentage_error(y_test, y_pred_test_gb)

print("\nPerformance Metrics:")
print(f"  Train R²: {train_r2_gb:.4f}")
print(f"  Test R²: {test_r2_gb:.4f}")
print(f"  Test MAE: ${test_mae_gb:,.0f}")
print(f"  Test RMSE: ${test_rmse_gb:,.0f}")
print(f"  Test MAPE: {test_mape_gb:.1%}")
print()

# Feature importance
feature_importance_gb = pd.DataFrame({
    'feature': all_rf_features,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features (by GB importance):")
print(feature_importance_gb.head(10).to_string(index=False))
print()

7. MODEL 4: GRADIENT BOOSTING

WHY GRADIENT BOOSTING:
  Business: Industry standard for pricing, high accuracy
  Technical: Sequential learning, corrects previous errors
  Use case: When we need best possible predictions

Training Gradient Boosting...

Performance Metrics:
  Train R²: 0.9783
  Test R²: 0.9687
  Test MAE: $49,395
  Test RMSE: $79,347
  Test MAPE: 18.2%

Top 10 Most Important Features (by GB importance):
                      feature  importance
            num__price_ma_30d    0.309557
         num__product_encoded    0.207839
             num__price_ma_7d    0.182052
         num__segment_encoded    0.108996
num__price_inventory_pressure    0.078832
   num__price_diff_competitor    0.026120
      num__is_summer_slowdown    0.013983
               num__qty_ma_7d    0.013827
              num__qty_ma_30d    0.012595
       num__is_academic_start    0.009419



In [10]:
# ============================================================================
# MODEL COMPARISON
# ============================================================================
print("="*80)
print("8. MODEL COMPARISON")
print("="*80)
print()

comparison = pd.DataFrame({
    'Model': ['Linear Regression', 'Ridge Regression', 'Random Forest', 'Gradient Boosting'],
    'Train_R²': [train_r2_lr, train_r2_ridge, train_r2_rf, train_r2_gb],
    'Test_R²': [test_r2_lr, test_r2_ridge, test_r2_rf, test_r2_gb],
    'Test_MAE': [test_mae_lr, test_mae_ridge, test_mae_rf, test_mae_gb],
    'Test_RMSE': [test_rmse_lr, test_rmse_ridge, test_rmse_rf, test_rmse_gb],
    'Test_MAPE_%': [test_mape_lr*100, test_mape_ridge*100, test_mape_rf*100, test_mape_gb*100]
})

comparison = comparison.round(4)
print(comparison.to_string(index=False))
print()

# Select best model
best_idx = comparison['Test_R²'].idxmax()
best_model_name = comparison.loc[best_idx, 'Model']
best_model_r2 = comparison.loc[best_idx, 'Test_R²']

print(f"🏆 BEST MODEL: {best_model_name}")
print(f"   Test R²: {best_model_r2:.4f}")
print(f"   Explains {best_model_r2*100:.1f}% of profit variation")
print()

# Business interpretation
print("BUSINESS INTERPRETATION:")
if best_model_name == 'Gradient Boosting':
    print("  ✓ Gradient Boosting wins - complex non-linear relationships captured")
    print("  ✓ Trade-off: Less interpretable than linear models")
    print("  ✓ Solution: Use SHAP values or feature importance for explainability")
    best_model = gb_model
elif best_model_name == 'Random Forest':
    print("  ✓ Random Forest wins - good balance of accuracy and speed")
    print("  ✓ Feature importance built-in for explainability")
    best_model = rf_model
else:
    print("  ✓ Linear model wins - simple relationships, highly interpretable")
    print("  ✓ Can directly explain coefficient impact")
    best_model = ridge_model if best_model_name == 'Ridge Regression' else lr_model
print()


8. MODEL COMPARISON

            Model  Train_R²  Test_R²    Test_MAE   Test_RMSE  Test_MAPE_%
Linear Regression    0.8636   0.8532 131590.0954 171930.7271     275.3054
 Ridge Regression    0.8636   0.8533 131553.1551 171869.0836     274.9159
    Random Forest    0.9609   0.9456  61487.6324 104623.2447      17.3943
Gradient Boosting    0.9783   0.9687  49394.7970  79346.6484      18.2402

🏆 BEST MODEL: Gradient Boosting
   Test R²: 0.9687
   Explains 96.9% of profit variation

BUSINESS INTERPRETATION:
  ✓ Gradient Boosting wins - complex non-linear relationships captured
  ✓ Trade-off: Less interpretable than linear models
  ✓ Solution: Use SHAP values or feature importance for explainability



 Since the Mean Absolute Error (MAE) was $\mathbf{\$49,395}$, the fact that the RMSE ($\$79,300$) is higher tells us that the model has a number of larger, isolated errors (outliers) that pull the RMSE up significantly beyond the MAE.

In [11]:
# ============================================================================
# CROSS-VALIDATION
# ============================================================================
print("="*80)
print("9. CROSS-VALIDATION (TIME SERIES)")
print("="*80)
print()

print("WHY CROSS-VALIDATION:")
print("  - Validate model stability across different time periods")
print("  - Detect overfitting")
print("  - Ensure model works on future data")
print()

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

print("Running 5-fold time series cross-validation on best model...")
cv_scores = cross_val_score(best_model, X_train_rfprocessed, y_train, cv=tscv, 
                             scoring='r2', n_jobs=-1)

print(f"\nCV R² Scores: {[f'{s:.4f}' for s in cv_scores]}")
print(f"Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
print()

if cv_scores.std() < 0.05:
    print("✓ Low variance across folds - model is stable")
else:
    print("⚠ High variance - model performance varies by time period")
print()


9. CROSS-VALIDATION (TIME SERIES)

WHY CROSS-VALIDATION:
  - Validate model stability across different time periods
  - Detect overfitting
  - Ensure model works on future data

Running 5-fold time series cross-validation on best model...

CV R² Scores: ['0.8583', '0.9571', '0.9638', '0.9625', '0.9652']
Mean CV R²: 0.9414 (+/- 0.0416)

✓ Low variance across folds - model is stable



In [12]:
# ============================================================================
# RESIDUAL ANALYSIS
# ============================================================================
print("="*80)
print("10. RESIDUAL ANALYSIS")
print("="*80)
print()

print("WHY RESIDUALS:")
print("  - Check for systematic errors (bias)")
print("  - Validate model assumptions")
print("  - Identify segments where model struggles")
print()

residuals = y_test - y_pred_test_gb

print("Residual Statistics:")
print(f"  Mean error: ${residuals.mean():,.0f} (should be ~0)")
print(f"  Median error: ${residuals.median():,.0f}")
print(f"  Std dev: ${residuals.std():,.0f}")
print()

# Check for bias by segment
df_test = df_sorted[split_idx:split_idx+len(y_test)].copy()
df_test['residual'] = residuals.values
df_test['abs_error'] = np.abs(residuals.values)

print("Mean Absolute Error by Product:")
product_errors = df_test.groupby('product')['abs_error'].mean().sort_values(ascending=False)
print(product_errors.apply(lambda x: f"${x:,.0f}"))
print()

print("BUSINESS INSIGHT:")
worst_product = product_errors.index[0]
best_product = product_errors.index[-1]
print(f"  Hardest to predict: {worst_product} (error: ${product_errors.iloc[0]:,.0f})")
print(f"  Easiest to predict: {best_product} (error: ${product_errors.iloc[-1]:,.0f})")
print(f"  → May need product-specific models or more features for {worst_product}")
print()


10. RESIDUAL ANALYSIS

WHY RESIDUALS:
  - Check for systematic errors (bias)
  - Validate model assumptions
  - Identify segments where model struggles

Residual Statistics:
  Mean error: $-1,379 (should be ~0)
  Median error: $-15
  Std dev: $79,355

Mean Absolute Error by Product:
product
Microscope     $97,722
Centrifuge     $86,129
PCR_System     $54,889
Reagent_Kit     $6,422
Pipettes        $6,271
Name: abs_error, dtype: object

BUSINESS INSIGHT:
  Hardest to predict: Microscope (error: $97,722)
  Easiest to predict: Pipettes (error: $6,271)
  → May need product-specific models or more features for Microscope



The key takeaway is that your model is accurate overall, but it reveals massive pricing volatility and risk in certain product segments.

1. Overall Confidence and RiskMean Error ($\$-1,379$): This means that on average, our model's predictions for the final profit are unbiased. We are not systematically over- or under-forecasting profit across the entire business. 

    Stakeholder takeaway: The model is fair and trustworthy.RMSE / Standard Deviation ($\approx \$79,300$): This represents our typical error or risk margin for any given deal.

    Interpretation: For a new, typical sale, we can expect our profit forecast to be off by $\mathbf{\$79,300}$ in either direction (up or down).

    Pricing Relevance: This is the size of the uncertainty buffer you should factor into large quotes or budgeting. Since the standard deviation ($\$79,300$) is much higher than the average error (MAE of $\approx \$49,400$), it confirms that a few deals have exceptionally high volatility, which leads us to the next point.

2. Pinpointing Pricing Volatility (The Product Breakdown) The breakdown of error by product is the most actionable part of this analysis for pricing. It tells you which products have consistent pricing and which have chaotic or highly customized pricing.

    A. Microscope and Centrifuge hasve MAE of around $\mathbf{\$90,000}$ signifing High Volatility & Pricing Chaos. The error is double the overall model's average error. This suggests sales or pricing for these products are highly inconsistent, heavily customized, or influenced by non-modeled factors (like deep discounts, special bundles, or competitor pricing not in your data).
    
        Action: This product requires a standardized pricing matrix or better feature tracking.

    B. Pipettes and Reagent Kits have MAE of aorund $\mathbf{\$6,300}$ signifing pricing Predictability & Consistency. The error is very low, meaning the factors driving Pipette  and Reagent kits profit are clear, consistent, and well-captured by the model. 
    
        Action: Trust the model's forecasts and pricing recommendations for these product lines.

In [13]:

# ============================================================================
# SAVE FINAL MODEL
# ============================================================================
print("="*80)
print("11. SAVE FINAL MODEL FOR PRODUCTION")
print("="*80)
print()

# Save model
joblib.dump(best_model, 'price_optimization_model.pkl')
print(f"✓ Saved: price_optimization_model.pkl ({best_model_name})")

# Save scaler
joblib.dump(preprocessor, 'feature_scaler_1.pkl')
print("✓ Saved: feature_scaler.pkl")

joblib.dump(rf_preprocessor, 'feature_scaler_rf.pkl')
print("✓ Saved: feature_scaler_rf.pkl")

# Save model metadata
model_metadata = {
    'model_type': best_model_name,
    'features': all_rf_features.tolist(),
    'features_expanded': all_features_expanded.tolist(),
    'target': target,
    'train_size': len(X_train),
    'test_size': len(X_test),
    'test_r2': float(best_model_r2),
    'test_mae': float(test_mae_gb),
    'test_rmse': float(test_rmse_gb),
    'cv_mean_r2': float(cv_scores.mean()),
    'cv_std_r2': float(cv_scores.std())
}

with open('model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)
print("✓ Saved: model_metadata.json")
print()



11. SAVE FINAL MODEL FOR PRODUCTION

✓ Saved: price_optimization_model.pkl (Gradient Boosting)
✓ Saved: feature_scaler.pkl
✓ Saved: feature_scaler_rf.pkl
✓ Saved: model_metadata.json



In [14]:
print("="*80)
print("MODEL BUILDING COMPLETE - PRODUCTION READY")
print("="*80)
print()

print("DELIVERABLES:")
print("  ✓ Trained model: price_optimization_model.pkl")
print("  ✓ Feature scaler: feature_scaler.pkl")
print("  ✓ Model metadata: model_metadata.json")
print("  ✓ Feature definitions: feature_metadata.json")
print()

print("MODEL PERFORMANCE:")
print(f"  Test R²: {best_model_r2:.4f} ({best_model_r2*100:.1f}% variance explained)")
print(f"  Test MAE: ${test_mae_gb:,.0f}")
print(f"  Test MAPE: {test_mape_gb:.1%}")
print()

print("NEXT STEPS:")
print("  1. Build Streamlit app for interactive optimization")
print("  2. Test with real pricing scenarios")
print("  3. A/B test recommendations vs current pricing")
print("  4. Monitor model performance in production")
print("  5. Retrain monthly with new data")
print()

print("="*80)

MODEL BUILDING COMPLETE - PRODUCTION READY

DELIVERABLES:
  ✓ Trained model: price_optimization_model.pkl
  ✓ Feature scaler: feature_scaler.pkl
  ✓ Model metadata: model_metadata.json
  ✓ Feature definitions: feature_metadata.json

MODEL PERFORMANCE:
  Test R²: 0.9687 (96.9% variance explained)
  Test MAE: $49,395
  Test MAPE: 18.2%

NEXT STEPS:
  1. Build Streamlit app for interactive optimization
  2. Test with real pricing scenarios
  3. A/B test recommendations vs current pricing
  4. Monitor model performance in production
  5. Retrain monthly with new data

